# [E-16]SessionBasedRecommendation 
_____

## 목차
### 1. 개요
    1.1 들어가기에 앞서
    1.2 루브릭 평가기준
 
### 2. 프로젝트 : Movielens 영화 SBR
    2.1 데이터 준비 및 전처리
    2.2 미니 배치의 구성
    2.3 모델 구성
    2.4 모델 학습
    2.5 모델 테스트
    
### 3. 결론
    3.1 결론
    3.2 참조
    3.3 회고
    
-----

## 1. 개요

### 1.1 들어가기에 앞서

#### Session-Parallel Mini-Batches 

![1](https://d3s0tskafalll9.cloudfront.net/media/images/input1.max-800x600.png)

- Session 1, 2, 3을 하나의 mini-batch로 만든다면, 이 미니 배치의 연산은 Session 3의 연산이 끝나야 끝나는 식

- 위 사진은 데이터 샘플 하나로 보고 mini-batch를 구성하여 input으로 넣는다면 길이가 제일 긴 세션의 연산이 끝날 때까지 짧은 세션들이 기다려야 한다는 문제점이 있음


![2](https://d3s0tskafalll9.cloudfront.net/media/images/input2.max-800x600.png)

- 위 문제점을 보완하도록, 위 사진처럼 **Session-Parallel Mini-Batches** 사용

- 이 방법은 Session이 끝날 때까지 기다리지 않고 병렬적으로 계산하는 방법.session2가 끝나면 session4가 시작하는 방식.



---

### 1.2 루브릭 평가기준

평가문항|상세기준
-|-
1. Movielens 데이터셋을 session based recommendation 관점으로 전처리하는 과정이 체계적으로 진행되었다.|데이터셋의 면밀한 분석을 토대로 세션단위 정의 과정(길이분석, 시간분석)을 합리적으로 수행한 과정이 기술되었다.
2. RNN 기반의 예측 모델이 정상적으로 구성되어 안정적으로 훈련이 진행되었다.|적절한 epoch만큼의 학습이 진행되는 과정에서 train loss가 안정적으로 감소하고, validation 단계에서의 Recall, MRR이 개선되는 것이 확인된다.
3. 세션정의, 모델구조, 하이퍼파라미터 등을 변경해서 실험하여 Recall, MRR 등의 변화추이를 관찰하였다.|3가지 이상의 변화를 시도하고 그 실험결과를 체계적으로 분석하였다.

---
## 2. 프로젝트 : Movielens 영화 SBR

### 2.1 데이터 준비 및 전처리

- 데이터셋 항목별 기본 분석, session length, session time, cleaning 등의 작업을 진행
- 데이터셋에서는 Session이 아닌 UserID 단위로 데이터가 생성되어 있으므로, 이를 Session 단위로 어떻게 해석할지 주의

##### (1) 데이터 준비

In [1]:
import datetime as dt
from pathlib import Path
import os

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import datetime, time

- 필요한 모듈 불러오기

In [2]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data/') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


- 주요 칼럼들이 포함된 데이터를 불러오기
- 데이터 개수: 1000209 개

---

##### (2) 평점 2점 이하의 데이터 제외

In [3]:
data = data[data['Rating'] > 2]

- 영화 추천 시스템이기 때문에 평점 2점 이하의 데이터를 제외

In [4]:
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


- 평점 2점 이하의 데이터를 제외한 데이터의 개수는 836478개

---

##### (3) Time 형식 변환

In [5]:
times = data["Time"] 
tmp_list = [] 
for time in times: 
    tmp_date = dt.datetime.fromtimestamp(time)
    tmp_list.append(tmp_date)
data["Time"] = tmp_list 
data

,UserId,ItemId,Rating,Time
31,1,3186,4,2000-12-31 22:00:19
22,1,1270,5,2000-12-31 22:00:55
27,1,1721,4,2000-12-31 22:00:55
37,1,1022,5,2000-12-31 22:00:55
24,1,2340,3,2000-12-31 22:01:43
...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29
999988,6040,1921,4,2001-08-10 14:41:04
1000172,6040,1784,3,2001-08-10 14:41:04
1000167,6040,161,3,2001-08-10 14:41:26


- Time 칼럼의 시간을 TImestamp에서 Datetime 형식으로 변환

---

##### (4) 데이터 개수 확인

In [6]:
data['UserId'].nunique(), data['ItemId'].nunique()

(6039, 3628)

- SessionId 대신 UserId 단위로 데이터가 생성되어 있으므로 변경
- UserId의 고유값: 6040
- ItemId의 고유값: 3706

In [7]:
session_length = data.groupby('UserId').size()
session_length

UserId
1        53
2       116
3        46
4        19
5       143
       ... 
6036    708
6037    189
6038     18
6039    119
6040    276
Length: 6039, dtype: int64

- session_length는 유저 각각의 영화에 대한 평점 개수를 의미

In [8]:
session_length.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
dtype: float64

- 데이터 요약을 위해 `describe()` 메서드 사용
- 요약본을 보고 확인할 수 있는 것은 평점기록의 최소는 1개 최대는 1968개, 평균은 약 138개

In [9]:
session_length.quantile(0.999)

1118.8860000000013

- 99.9% 세션의 길이가 약 1118 이하인데, 최대 길이와의 격차가 심함

In [10]:
long_session = session_length[session_length==1968].index[0]
data[data['UserId']==long_session]

,UserId,ItemId,Rating,Time
696969,4169,1268,5,2000-08-03 20:09:52
697168,4169,2617,4,2000-08-03 20:09:52
697185,4169,2628,4,2000-08-03 20:09:52
697219,4169,2653,4,2000-08-03 20:09:52
697275,4169,423,3,2000-08-03 20:09:52
...,...,...,...,...
697055,4169,3207,3,2002-06-15 20:23:26
695958,4169,3413,3,2002-06-15 20:33:11
695702,4169,1413,3,2002-06-15 21:03:51
697358,4169,494,4,2002-06-15 21:16:00


- 최대길이(최다 평점)의 유저 데이터를 살펴보았으나, 초기에 1초의 격차도 없이 평점을 매긴 기록이 있음. 
- 추후 최근 데이터만을 다룰 것이기에 제거하지 않고 포함

---

##### (5) Session Time

In [11]:
oldest, latest = data['Time'].min(), data['Time'].max()
print(oldest) 
print(latest)

2000-04-25 23:05:32
2003-02-28 17:49:50


- 데이터가 발생한 시점이 2000년대 극초반이고 약 3년치의 데이터 포함

In [12]:
year_ago = latest - dt.timedelta(730)      
data = data[data['Time'] > year_ago]   
data

,UserId,ItemId,Rating,Time
2327,19,318,4,2001-07-08 01:43:18
2492,19,1234,5,2001-07-08 01:43:56
2503,20,1694,3,2001-12-29 23:37:51
2512,20,1468,3,2001-12-29 23:37:51
2517,20,2858,4,2001-12-29 23:37:51
...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29
999988,6040,1921,4,2001-08-10 14:41:04
1000172,6040,1784,3,2001-08-10 14:41:04
1000167,6040,161,3,2001-08-10 14:41:26


- 최대한 최근 데이터를 반영하기 위해 최근 데이터의 1년치만 확인

---

##### (6) Train / Valid / Test split

In [13]:
def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['Time'].max()
    session_last_time = data.groupby('UserId')['Time'].max()
    session_in_train = session_last_time[session_last_time < final_time - dt.timedelta(n_days)].index
    session_in_test = session_last_time[session_last_time >= final_time - dt.timedelta(n_days)].index

    before_date = data[data['UserId'].isin(session_in_train)]
    after_date = data[data['UserId'].isin(session_in_test)]
    after_date = after_date[after_date['ItemId'].isin(before_date['ItemId'])]
    return before_date, after_date

In [14]:
tr, test = split_by_date(data, n_days=30)
tr, val = split_by_date(tr, n_days=30)

- 가장 마지막 30일 기간 동안을 Test로, 30일 전부터 30일 전 가지를 valid set으로 나눔

In [15]:
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Sessions: {data["UserId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["Time"].min()}\n'
          f'\t Last Time : {data["Time"].max()}\n')

In [16]:
stats_info(tr, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 37482
	 Sessions: 790
	 Items: 2950
	 First Time : 2001-02-28 19:49:34
	 Last Time : 2002-12-30 02:26:14

* valid Set Stats Info
	 Events: 6858
	 Sessions: 81
	 Items: 2062
	 First Time : 2001-02-28 19:06:53
	 Last Time : 2003-01-29 03:00:40

* test Set Stats Info
	 Events: 12274
	 Sessions: 98
	 Items: 2457
	 First Time : 2001-03-01 03:10:09
	 Last Time : 2003-02-28 17:49:50



In [17]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [18]:
save_path = data_path / 'processed'
save_path.mkdir(parents=True, exist_ok=True)

tr.to_pickle(save_path / 'train.pkl')
val.to_pickle(save_path / 'valid.pkl')
test.to_pickle(save_path / 'test.pkl')

---

### 2.2 미니 배치의 구성

실습 코드 내역을 참고하여 데이터셋과 미니 배치를 구성해 봅시다. Session-Parallel Mini-Batch의 개념에 따라, 학습 속도의 저하가 최소화될 수 있도록 구성합니다.
단, 위 Step 1에서 Session 단위를 어떻게 정의했느냐에 따라서 Session-Parallel Mini-Batch이 굳이 필요하지 않을 수도 있습니다.

In [19]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['UserId'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['UserId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('UserId').size().cumsum()
        return offsets

- 데이터가 주어지면 세션이 시작되는 인덱스를 담 값과 세션을 새로 인덱싱한 값을 갖는 클래스 생성

In [20]:
tr_dataset = SessionDataset(tr)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,Time,item_idx
2327,19,318,4,2001-07-08 01:43:18,0
2492,19,1234,5,2001-07-08 01:43:56,1
2503,20,1694,3,2001-12-29 23:37:51,2
2512,20,1468,3,2001-12-29 23:37:51,3
2517,20,2858,4,2001-12-29 23:37:51,4
2504,20,2641,4,2001-12-29 23:38:35,5
2510,20,1375,3,2001-12-29 23:38:35,6
2520,20,3753,5,2001-12-29 23:38:36,7
2507,20,3527,4,2001-12-29 23:39:41,8
2511,20,1527,5,2001-12-29 23:39:41,9


- train데이터로 `SessionDataset` 객체를 생성후 인스턴스 변수 확인

In [21]:
tr_dataset.click_offsets

array([    0,     2,    12,    22,    41,    46,   119,   179,   259,
         286,   288,   322,   341,   342,   348,   354,   551,   696,
         698,   728,   738,   755,   759,   770,   834,   847,   851,
         861,  1007,  1047,  1182,  1190,  1418,  1421,  1448,  1482,
        1488,  1507,  1844,  1858,  1929,  2001,  2052,  2080,  2162,
        2165,  2179,  2300,  2396,  2404,  2427,  2433,  2489,  2490,
        2514,  2515,  2520,  2529,  2582,  2596,  2616,  2639,  2662,
        2710,  2721,  2956,  2968,  2972,  3008,  3157,  3171,  3178,
        3369,  3382,  3554,  3698,  3702,  3703,  3714,  3745,  3754,
        3757,  3761,  3796,  3970,  3982,  3994,  4001,  4084,  4110,
        4163,  4202,  4214,  4270,  4271,  4287,  4317,  4520,  4544,
        4559,  4565,  4615,  4624,  4638,  4642,  4645,  4651,  4652,
        4667,  4729,  4785,  4786,  4823,  4844,  4921,  4953,  4961,
        5079,  5204,  5243,  5345,  5466,  5557,  5575,  5581,  5629,
        5696,  5813,

- `click_offsets` 변수는 각 세션이 시작된 인덱스를 포함

In [22]:
tr_dataset.session_idx

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

- `session_idx` 변수는 각 세션을 인덱싱한 `np.array`

In [23]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):  
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.
        
        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]  
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i  
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished

In [24]:
tr_data_loader = SessionDataLoader(tr_dataset, batch_size=4)
tr_dataset.df.head(15)

,UserId,ItemId,Rating,Time,item_idx
2327,19,318,4,2001-07-08 01:43:18,0
2492,19,1234,5,2001-07-08 01:43:56,1
2503,20,1694,3,2001-12-29 23:37:51,2
2512,20,1468,3,2001-12-29 23:37:51,3
2517,20,2858,4,2001-12-29 23:37:51,4
2504,20,2641,4,2001-12-29 23:38:35,5
2510,20,1375,3,2001-12-29 23:38:35,6
2520,20,3753,5,2001-12-29 23:38:36,7
2507,20,3527,4,2001-12-29 23:39:41,8
2511,20,1527,5,2001-12-29 23:39:41,9


In [25]:
iter_ex = iter(tr_data_loader)

In [26]:
inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [ 0  2 12 21]
Label Item Idx are :       [ 1  3 13 22]
Previous Masked Input Idx are []


---

### 2.3 모델 구성

##### (1) Evaluation Metric

In [27]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

---

##### (2) Model Architecture

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [29]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [30]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['UserId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

args = Args(tr, val, test, batch_size=8, hsz=50, drop_rate=0.1, lr=0.001, epochs=10, k=20)

In [31]:
model = create_model(args)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(8, 1, 2950)]            0         
_________________________________________________________________
GRU (GRU)                    [(8, 50), (8, 50)]        450300    
_________________________________________________________________
dropout (Dropout)            (8, 50)                   0         
_________________________________________________________________
dense (Dense)                (8, 2950)                 150450    
Total params: 600,750
Trainable params: 600,750
Non-trainable params: 0
_________________________________________________________________


---

### 2.4 모델 학습

In [32]:
# train 셋으로 학습하면서 valid 셋으로 검증합니다.
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr['UserId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  # valid set에 대해 검증합니다.

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다. 
                                             # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data['UserId'].nunique()
    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(pred, direction='DESCENDING')  # softmax 값이 큰 순서대로 sorting 합니다.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

In [33]:
train_model(model, args)

Evaluation:  91%|█████████ | 768/847 [00:48<00:04, 15.85it/s]


	 - Recall@20 epoch 1: 0.057780
	 - MRR@20    epoch 1: 0.014595



Evaluation:  91%|█████████ | 768/847 [00:48<00:04, 15.88it/s]


	 - Recall@20 epoch 2: 0.073079
	 - MRR@20    epoch 2: 0.016979



Evaluation:  91%|█████████ | 768/847 [00:48<00:04, 15.99it/s]


...

	 - Recall@20 epoch 8: 0.108724
	 - MRR@20    epoch 8: 0.033206



Evaluation:  91%|█████████ | 768/847 [00:47<00:04, 16.24it/s]


	 - Recall@20 epoch 9: 0.108724
	 - MRR@20    epoch 9: 0.033469



Evaluation:  91%|█████████ | 768/847 [00:47<00:04, 16.22it/s]

	 - Recall@20 epoch 10: 0.102865
	 - MRR@20    epoch 10: 0.032629



In [34]:
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

test_model(model, args, test)

Evaluation:  96%|█████████▌| 1459/1522 [01:31<00:03, 16.02it/s]

	 - Recall@20: 0.075223
	 - MRR@20: 0.021859



---

## 3. 결론

### 3.1 결론

첫 시도때 배치 사이즈 2048부터 256까지, 오류가 발생하며 훈련이 진행되지 않는 문제점이 있었다.

메모리문제인 것으로 생각이 되나 배치사이즈 128부터는 모델 훈련이 가능했다.

또 다른 문제가 있었다면, Train 학습이 100%까지 되지 않고 게이지가 끝까지 차지 않는 상황이었다.

그래서 배치 사이즈를 128부터 8까지 낮추며 훈련을 진행했다.

![](https://images.velog.io/images/khkk4953/post/8fbc5db4-933a-424b-b142-6ecf1eb687b9/16-1.png)

![](https://images.velog.io/images/khkk4953/post/ee28c508-5829-48d5-b5a2-9a5be7536d31/16-2.png)

![](https://images.velog.io/images/khkk4953/post/b7ced391-b47b-4e98-8d1d-a8f650ced52c/16-3.png)

![](https://images.velog.io/images/khkk4953/post/81718a80-b12c-48f4-8986-13f2da57036f/16-4.png)

![](https://images.velog.io/images/khkk4953/post/fe04b365-524f-44a7-832a-6f7a0bb383e1/16-5.png)

점차적으로 배치 사이즈를 낮추면 낮출 수록 게이지가 더 차올랐다. 하이퍼파라미터를 변경하며 유의미한 지표 값에 맞추는 것이 목적이었으나, 그냥 훈련게이지를 채우는 것이 목적이 되어버렸던 것 같다.(?)

위 결과들로 보았을 때, `Recall@k` 지표와 `MRR@k`가 128부터 32까지는 배치 사이즈를 낮추면 감소하는 경향이 있었으나, 32에서 16으로 낮췄을 때 지표들의 값들이 향상했다. 또한 배치 사이즈가 8으로 낮췄을 때 에포크를 10으로 변경했더니, 그 전보단 조금 유의미한 지표 값들이 결정된 것 같았다. 

---

### 3.2 참조

- Time 변환 코드: https://github.com/Jeonda/aiffel/blob/master/%5BE17%5DMovielens_SBR.ipynb

- 평가지표: https://zzaebok.github.io/recommender_system/metrics/rec_metrics/

---

### 3.3 회고

- 추천시스템은 정말 흥미로운 주제인 것 같다. 노드를 진행하는 것에 있어 지루함은 없었다. 다만 평가지표에 대한 판단이 어렵고 힘들게 느껴졌다. 대표적으로 정확도 같은 지표는 추천시스템에 적용하기 어렵다고 한다. 추천 시스템에서 아이템을 얼마나 상위권에 잘 올라갔는지, 사용자에게 있어 추천된 아이템 간의 상대적인 선호도가 잘 반영이 되었는지에 만족하는 metric을 찾아야하기 때문이다. 본 노드에서는 `Recall@k` 지표와 `MRR@k`가 사용되었는데, `Recall@k`는 전체 relevant한 아이템 중 추천된 아이템이 속한 비율을 말하고,`MRR@k`는 첫 번째로 등장하는 relevant한 아이템이 우리의 추천상 몇 번째에 위치하는지를 나타내는 지표를 뜻한다. 그리고 @ 뒤에 붙은 k가 추천된 개수를 의미한다. 실제 훈련의 결과 지표들은 유의미한 결과를 가져오지 못했는데, 이런 지표들을 자주 사용하지 못하다보니 명백히 그 기준을 정하기가 애매한 것 같다. 평소 훈련 과정에만 신경을 썼지 훈련 평가에 대해서도 실력이 미비하다는 생각이 들었다. 그렇기에 다양한 지표들을 사용할 줄 알고, 그 기준을 결정할 줄 아는 연습이 필요할 것 같다.